In [16]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from Funcoes_Comuns import avaliar_modelo

In [17]:
def permutation_test_mae(y_true, y_pred_model1, y_pred_model2, n_permutations=10000):
    """
    Realiza um teste de permutação para comparar duas previsões de modelo usando o MAE.
    Verifica se o modelo 2 é estatisticamente superior ao modelo 1 de referencia. 

    Args:
        y_true (array-like): Valores reais do conjunto de teste.
        y_pred_model1 (array-like): Previsões do Modelo 1 (Modelo referência).
        y_pred_model2 (array-like): Previsões do Modelo 2 (Modelo a ser verificado se é superior).
        n_permutations (int): Número de permutações a serem realizadas.

    Returns:
        tuple: (p_value, observed_mae_diff)
               p_value: O p-valor do teste.
               observed_mae_diff: A diferença observada de MAE (MAE_model1 - MAE_model2).
                                  Um valor positivo indica que o Modelo 2 é melhor.
    """

    # 1. Calcular a diferença observada no MAE
    mae_model1 = mean_absolute_error(y_true, y_pred_model1)
    mae_model2 = mean_absolute_error(y_true, y_pred_model2)
    observed_mae_diff = mae_model1 - mae_model2 # Se positivo, significa que o Modelo 2 tem MAE menor

    permutation_diffs = []
    n_samples = len(y_true)

    # Obter os erros absolutos para cada ponto de dados e cada modelo
    abs_errors_model1 = np.abs(y_true - y_pred_model1)
    abs_errors_model2 = np.abs(y_true - y_pred_model2)

    for _ in range(n_permutations):
        # Cria uma máscara booleana para decidir quais erros absolutos "trocar"
        # entre os modelos para simular a hipótese nula (onde os erros vêm da mesma distribuição).
        swap_mask = np.random.rand(n_samples) < 0.5 # 50% de chance de trocar para cada ponto

        # Cria cópias dos erros absolutos para a permutação
        permuted_errors_model1 = np.copy(abs_errors_model1)
        permuted_errors_model2 = np.copy(abs_errors_model2)

        # Realiza a troca dos erros absolutos com base na máscara
        permuted_errors_model1[swap_mask] = abs_errors_model2[swap_mask]
        permuted_errors_model2[swap_mask] = abs_errors_model1[swap_mask]

        # Recalcula o MAE para os conjuntos de erros permutados
        perm_mae_model1 = np.mean(permuted_errors_model1)
        perm_mae_model2 = np.mean(permuted_errors_model2)

        # Calcula a diferença permutada
        perm_diff = perm_mae_model1 - perm_mae_model2
        permutation_diffs.append(perm_diff)

    permutation_diffs = np.array(permutation_diffs)

    # 4. Calcular o p-valor
    # Estamos testando se a diferença observada (MAE_model1 - MAE_model2) é POSITIVA
    # e significativa (ou seja, se MAE_model2 é significativamente menor que MAE_model1).
    # O p-valor é a proporção de diferenças permutadas que são maiores ou iguais à diferença observada.
    p_value = (np.sum(permutation_diffs >= observed_mae_diff) + 1) / (n_permutations + 1)

    return p_value, observed_mae_diff

In [30]:
def permutation_test_rmse(y_true, y_pred_model1, y_pred_model2, n_permutations=10000):
    """
    Realiza um teste de permutação para comparar duas previsões de modelo usando o RMSE.
    Verifica se o modelo 2 é estatisticamente superior ao modelo 1 de referencia.

    Args:
        y_true (array-like): Valores reais do conjunto de teste.
        y_pred_model1 (array-like): Previsões do Modelo 1 (referência).
        y_pred_model2 (array-like): Previsões do Modelo 2 (candidato).
        n_permutations (int): Número de permutações a serem realizadas.

    Returns:
        tuple: (p_value, observed_rmse_diff)
               p_value: O p-valor do teste.
               observed_rmse_diff: Diferença observada de RMSE (RMSE_model1 - RMSE_model2).
                                   Valor positivo indica que o Modelo 2 é melhor.
    """
    # 1. Calcular a diferença observada no RMSE
    rmse_model1 = np.sqrt(np.mean((y_true - y_pred_model1) ** 2))
    rmse_model2 = np.sqrt(np.mean((y_true - y_pred_model2) ** 2))
    observed_rmse_diff = rmse_model1 - rmse_model2

    permutation_diffs = []
    n_samples = len(y_true)

    # Obter os erros quadráticos para cada ponto de dados e cada modelo
    sq_errors_model1 = (y_true - y_pred_model1) ** 2
    sq_errors_model2 = (y_true - y_pred_model2) ** 2

    for _ in range(n_permutations):
        swap_mask = np.random.rand(n_samples) < 0.5

        permuted_errors_model1 = np.copy(sq_errors_model1)
        permuted_errors_model2 = np.copy(sq_errors_model2)

        permuted_errors_model1[swap_mask] = sq_errors_model2[swap_mask]
        permuted_errors_model2[swap_mask] = sq_errors_model1[swap_mask]

        perm_rmse_model1 = np.sqrt(np.mean(permuted_errors_model1))
        perm_rmse_model2 = np.sqrt(np.mean(permuted_errors_model2))

        perm_diff = perm_rmse_model1 - perm_rmse_model2
        permutation_diffs.append(perm_diff)

    permutation_diffs = np.array(permutation_diffs)
    p_value = (np.sum(permutation_diffs >= observed_rmse_diff) + 1) / (n_permutations + 1)

    return p_value, observed_rmse_diff

In [18]:
# Ler base unificada do ENEM 2023 + Censo Escolar 2023
df_unificado = pd.read_pickle('Bases\Finais\enem_censo_2023_full.pkl')

# 5 Variáveis alvo (origem ENEM)
# 40 Variáveis preditoras socioeconômicas (origem ENEM)
# 66 Variáveis preditoras escolares (origem ENEM e Censo Escolar)

In [19]:
colunas_alvo = ['NUM_NOTA_CH', 'NUM_NOTA_CN', 'NUM_NOTA_LC', 'NUM_NOTA_MT', 'NUM_NOTA_REDACAO']

colunas_microdados_enem = [
    "BIN_Q001_DUMMY_H", "BIN_Q002_DUMMY_H", "BIN_Q018",
    "BIN_Q020", "BIN_Q021", "BIN_Q023", "BIN_Q025", "CAT_COR_RACA", "CAT_CO_MUNICIPIO_ESC", 
    "CAT_CO_UF_ESC", "CAT_DEPENDENCIA_ADM_ESC", "CAT_ENSINO", "CAT_ESCOLA", "CAT_ESTADO_CIVIL", "CAT_FAIXA_ETARIA",
    "CAT_LINGUA", "CAT_LOCALIZACAO_ESC", "CAT_NACIONALIDADE", "CAT_Q003", "CAT_Q004", "CAT_SEXO", "CAT_SIT_FUNC_ESC",
    "NUM_Q001", "NUM_Q002", "NUM_Q005", "NUM_Q006", "NUM_Q007", "NUM_Q008", "NUM_Q009", "NUM_Q010", "NUM_Q011",
    "NUM_Q012", "NUM_Q013", "NUM_Q014", "NUM_Q015", "NUM_Q016", "NUM_Q017", "NUM_Q019", "NUM_Q022", "NUM_Q024"
]

colunas_socioeconomicas = [
    "BIN_Q001_DUMMY_H", "BIN_Q002_DUMMY_H", "BIN_Q018", "BIN_Q020", "BIN_Q021", "BIN_Q023", "BIN_Q025", "CAT_COR_RACA",
    "CAT_ESTADO_CIVIL", "CAT_FAIXA_ETARIA", "CAT_LINGUA", "CAT_NACIONALIDADE", "CAT_Q003", "CAT_Q004", "CAT_SEXO",
    "NUM_Q001", "NUM_Q002", "NUM_Q005", "NUM_Q006", "NUM_Q007", "NUM_Q008", "NUM_Q009", "NUM_Q010", "NUM_Q011",
    "NUM_Q012", "NUM_Q013", "NUM_Q014", "NUM_Q015", "NUM_Q016", "NUM_Q017", "NUM_Q019", "NUM_Q022", "NUM_Q024"
]

In [20]:
# separar em treino e teste
X = df_unificado.drop(columns=colunas_alvo)
y = df_unificado['NUM_NOTA_CH']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# Converter colunas inteiras para float
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

In [22]:
# Selecionar apenas as colunas socioeconômicas
X_train_socio = X_train[colunas_socioeconomicas]
X_test_socio = X_test[colunas_socioeconomicas]

X_train_enem = X_train[colunas_microdados_enem]
X_test_enem = X_test[colunas_microdados_enem]

In [23]:
# Carregar modelo treinado socioeconômico + escolar
with open('Modelos\modelo_lgbm_bayes_censo_enem.pkl', 'rb') as file:
    modelo_unificado = joblib.load(file)

# Carregar modelo treinado socioeconômico
with open('Modelos\modelo_lgbm_bayes_socioeconomico.pkl', 'rb') as file:
    modelo_socio = joblib.load(file)

# Carregar modelo treinado ENEM
with open('Modelos\modelo_lgbm_bayes.pkl', 'rb') as file:
    modelo_enem = joblib.load(file)

In [24]:
# Aplicar o modelo nos dados de teste
print("Aplicando modelos nos dados de teste...")

y_pred_unificado = modelo_unificado.predict(X_test)
print("Modelo unificado aplicado com sucesso.")

y_pred_socio = modelo_socio.predict(X_test_socio)
print("Modelo socioeconômico aplicado com sucesso.")

y_pred_enem = modelo_enem.predict(X_test_enem)
print("Modelo ENEM aplicado com sucesso.")

Aplicando modelos nos dados de teste...
Modelo unificado aplicado com sucesso.
Modelo socioeconômico aplicado com sucesso.
Modelo ENEM aplicado com sucesso.


In [33]:
# Avaliação grupo teste
print("Avaliação dos modelos nos dados de teste...\n")

print("Modelo unificado:")
avaliar_modelo(y_test, y_pred_unificado, "teste")
print("")
print("Modelo socioeconômico:")
avaliar_modelo(y_test, y_pred_socio, "teste")
print("")
print("Modelo ENEM:")
avaliar_modelo(y_test, y_pred_enem, "teste")

Avaliação dos modelos nos dados de teste...

Modelo unificado:
MAE (teste): 54.8861
RMSE (teste): 69.6043
R2 (teste): 0.3204

Modelo socioeconômico:
MAE (teste): 56.5698
RMSE (teste): 71.5463
R2 (teste): 0.2820

Modelo ENEM:
MAE (teste): 54.8689
RMSE (teste): 69.6190
R2 (teste): 0.3201


    y_test: seus valores reais do conjunto de teste
    y_pred_modelo_referencia: previsões do seu modelo de referência
    y_pred_modelo_candidato: previsões do seu modelo candidato a melhoria

Testes modelo Socioeconômico vs Unificado

In [ ]:
# Defina os nomes dos modelos para a comparação
nome_modelo_referencia = "Socioeconômico"
nome_modelo_candidato = "Unificado"

# Defina as previsões dos modelos a serem comparados
y_pred_modelo_referencia = y_pred_socio
y_pred_modelo_candidato = y_pred_unificado
alpha = 0.001 # Nível de significância para o teste

MAE

In [ ]:
print(f"Executando Teste de Permutação para MAE entre '{nome_modelo_referencia}' (referência) e '{nome_modelo_candidato}' (candidato)...")
p_value, obs_diff = permutation_test_mae(
    y_test,
    y_pred_modelo_referencia,
    y_pred_modelo_candidato,
    n_permutations=10000
)

# Exibir os resultados
mae_ref = mean_absolute_error(y_test, y_pred_modelo_referencia)
mae_cand = mean_absolute_error(y_test, y_pred_modelo_candidato)

print(f"\nMAE do Modelo {nome_modelo_referencia}: {mae_ref:.4f}")
print(f"MAE do Modelo {nome_modelo_candidato}: {mae_cand:.4f}")
print(f"Diferença observada (MAE_{nome_modelo_referencia} - MAE_{nome_modelo_candidato}): {obs_diff:.4f}")
print(f"P-valor: {p_value:.4f}")

# Interpretação do resultado
if p_value < alpha:
    print(f"\nCom p-valor {p_value:.4f} < {alpha}, rejeitamos a hipótese nula.")
    print(f"A melhora observada no MAE com o modelo '{nome_modelo_candidato}' é estatisticamente significativa.")
else:
    print(f"\nCom p-valor {p_value:.4f} >= {alpha}, falhamos em rejeitar a hipótese nula.")
    print(f"A melhora observada no MAE com o modelo '{nome_modelo_candidato}' NÃO é estatisticamente significativa (pode ser devido ao acaso).")

Executando Teste de Permutação para MAE entre 'Socioeconômico' (referência) e 'Unificado' (candidato)...



MAE do Modelo Socioeconômico: 56.5698
MAE do Modelo Unificado: 54.8861
Diferença observada (MAE_Socioeconômico - MAE_Unificado): 1.6837
P-valor: 0.0001

Com p-valor 0.0001 < 0.001, rejeitamos a hipótese nula.
A melhora observada no MAE com o modelo 'Unificado' é estatisticamente significativa.


RMSE

In [ ]:
print(f"Executando Teste de Permutação para RMSE entre '{nome_modelo_referencia}' (referência) e '{nome_modelo_candidato}' (candidato)...")
p_value, obs_diff = permutation_test_rmse(
    y_test,
    y_pred_modelo_referencia,
    y_pred_modelo_candidato,
    n_permutations=10000
)

# Exibir os resultados
rmse_ref = np.sqrt(np.mean((y_test - y_pred_modelo_referencia) ** 2))
rmse_cand = np.sqrt(np.mean((y_test - y_pred_modelo_candidato) ** 2))

print(f"\nRMSE do Modelo {nome_modelo_referencia}: {rmse_ref:.4f}")
print(f"RMSE do Modelo {nome_modelo_candidato}: {rmse_cand:.4f}")
print(f"Diferença observada (RMSE_{nome_modelo_referencia} - RMSE_{nome_modelo_candidato}): {obs_diff:.4f}")
print(f"P-valor: {p_value:.4f}")

# Interpretação do resultado
if p_value < alpha:
    print(f"\nCom p-valor {p_value:.4f} < {alpha}, rejeitamos a hipótese nula.")
    print(f"A melhora observada no RMSE com o modelo '{nome_modelo_candidato}' é estatisticamente significativa.")
else:
    print(f"\nCom p-valor {p_value:.4f} >= {alpha}, falhamos em rejeitar a hipótese nula.")
    print(f"A melhora observada no RMSE com o modelo '{nome_modelo_candidato}' NÃO é estatisticamente significativa (pode ser devido ao acaso).")

Executando Teste de Permutação para RMSE entre 'Socioeconômico' (referência) e 'Unificado' (candidato)...

RMSE do Modelo Socioeconômico: 71.5463
RMSE do Modelo Unificado: 69.6043
Diferença observada (RMSE_Socioeconômico - RMSE_Unificado): 1.9421
P-valor: 0.0001

Com p-valor 0.0001 < 0.001, rejeitamos a hipótese nula.
A melhora observada no RMSE com o modelo 'Unificado' é estatisticamente significativa.


Testes modelo ENEM vs Unificado

In [ ]:
# Defina os nomes dos modelos para a comparação
nome_modelo_referencia = "Microdados ENEM"
nome_modelo_candidato = "Unificado"

# Defina as previsões dos modelos a serem comparados
y_pred_modelo_referencia = y_pred_enem
y_pred_modelo_candidato = y_pred_unificado
alpha = 0.001 # Nível de significância para o teste

MAE

In [ ]:
print(f"Executando Teste de Permutação para MAE entre '{nome_modelo_referencia}' (referência) e '{nome_modelo_candidato}' (candidato)...")
p_value, obs_diff = permutation_test_mae(
    y_test,
    y_pred_modelo_referencia,
    y_pred_modelo_candidato,
    n_permutations=10000
)

# Exibir os resultados
mae_ref = mean_absolute_error(y_test, y_pred_modelo_referencia)
mae_cand = mean_absolute_error(y_test, y_pred_modelo_candidato)

print(f"\nMAE do Modelo {nome_modelo_referencia}: {mae_ref:.4f}")
print(f"MAE do Modelo {nome_modelo_candidato}: {mae_cand:.4f}")
print(f"Diferença observada (MAE_{nome_modelo_referencia} - MAE_{nome_modelo_candidato}): {obs_diff:.4f}")
print(f"P-valor: {p_value:.4f}")

# Interpretação do resultado
if p_value < alpha:
    print(f"\nCom p-valor {p_value:.4f} < {alpha}, rejeitamos a hipótese nula.")
    print(f"A melhora observada no MAE com o modelo '{nome_modelo_candidato}' é estatisticamente significativa.")
else:
    print(f"\nCom p-valor {p_value:.4f} >= {alpha}, falhamos em rejeitar a hipótese nula.")
    print(f"A melhora observada no MAE com o modelo '{nome_modelo_candidato}' NÃO é estatisticamente significativa (pode ser devido ao acaso).")

Executando Teste de Permutação para MAE entre 'Microdados ENEM' (referência) e 'Unificado' (candidato)...

MAE do Modelo Microdados ENEM: 54.8689
MAE do Modelo Unificado: 54.8861
Diferença observada (MAE_Microdados ENEM - MAE_Unificado): -0.0172
P-valor: 1.0000

Com p-valor 1.0000 >= 0.001, falhamos em rejeitar a hipótese nula.
A melhora observada no MAE com o modelo 'Unificado' NÃO é estatisticamente significativa (pode ser devido ao acaso).


RMSE

In [ ]:
print(f"Executando Teste de Permutação para RMSE entre '{nome_modelo_referencia}' (referência) e '{nome_modelo_candidato}' (candidato)...")
p_value, obs_diff = permutation_test_rmse(
    y_test,
    y_pred_modelo_referencia,
    y_pred_modelo_candidato,
    n_permutations=10000
)

# Exibir os resultados
rmse_ref = np.sqrt(np.mean((y_test - y_pred_modelo_referencia) ** 2))
rmse_cand = np.sqrt(np.mean((y_test - y_pred_modelo_candidato) ** 2))

print(f"\nRMSE do Modelo {nome_modelo_referencia}: {rmse_ref:.4f}")
print(f"RMSE do Modelo {nome_modelo_candidato}: {rmse_cand:.4f}")
print(f"Diferença observada (RMSE_{nome_modelo_referencia} - RMSE_{nome_modelo_candidato}): {obs_diff:.4f}")
print(f"P-valor: {p_value:.4f}")

# Interpretação do resultado
if p_value < alpha:
    print(f"\nCom p-valor {p_value:.4f} < {alpha}, rejeitamos a hipótese nula.")
    print(f"A melhora observada no RMSE com o modelo '{nome_modelo_candidato}' é estatisticamente significativa.")
else:
    print(f"\nCom p-valor {p_value:.4f} >= {alpha}, falhamos em rejeitar a hipótese nula.")
    print(f"A melhora observada no RMSE com o modelo '{nome_modelo_candidato}' NÃO é estatisticamente significativa (pode ser devido ao acaso).")

Executando Teste de Permutação para RMSE entre 'Microdados ENEM' (referência) e 'Unificado' (candidato)...

RMSE do Modelo Microdados ENEM: 69.6190
RMSE do Modelo Unificado: 69.6043
Diferença observada (RMSE_Microdados ENEM - RMSE_Unificado): 0.0147
P-valor: 0.0001

Com p-valor 0.0001 < 0.001, rejeitamos a hipótese nula.
A melhora observada no RMSE com o modelo 'Unificado' é estatisticamente significativa.
